In [1]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint

#Gensim & nltk
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models
from gensim.models import HdpModel
#from gensim.parsing.preprocessing import STOPWORDS
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, SnowballStemmer  
from nltk.stem.porter import *
from nltk.stem.lancaster import LancasterStemmer
from gensim.parsing.preprocessing import strip_numeric

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Upendra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
stop_words ={'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very','|','advertisement','new','york','times','would','m','re','mr','ms','look','say','see','put','take','told','might','well','go','still','made','much','way','u','us','like','ve','get','said','also','could','use','next','week','going','according','back','since','around','far','used','less','often','using','d','make','need','many','one','two','three','come','among','another','last','says','based','later','start','even','odd','o','dr','across','really','every','year','years','think','know','percent','want','called','city','right','including','high','part','good','day','life','big','news','help','group','long',
'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than','-',';','via'}

In [3]:
def lemmatize_stemming(text):
    lancaster_stemmer = LancasterStemmer()
    return lancaster_stemmer.stem(text)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        try:
            token=re.sub(" \d+", " ", token)
            if token not in stop_words and token.isalpha() and len(token) > 3:
                result.append(token)
        except Exception:
            pass
    return result

In [4]:
data1 = pd.read_csv('redditPost.csv')
data2 = pd.read_excel('comment_reddit.xlsx')
frames = [data1.title, data2.Comments]
result = pd.concat(frames)
processed_docs = result.map(preprocess)
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=10, no_above=0.5, )#keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [5]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1400,
                                           passes=1,
                                           alpha='auto',
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.033*"consulting" + 0.018*"time" + 0.017*"gmat" + 0.017*"interview" + '
  '0.016*"tech" + 0.013*"people" + 0.013*"experience" + 0.011*"work" + '
  '0.011*"company" + 0.010*"full"'),
 (1,
  '0.023*"practice" + 0.020*"gmat" + 0.018*"tech" + 0.017*"months" + '
  '0.017*"didn" + 0.017*"west" + 0.014*"coast" + 0.014*"tests" + 0.013*"work" '
  '+ 0.013*"anderson"'),
 (2,
  '0.031*"school" + 0.027*"work" + 0.024*"thanks" + 0.021*"experience" + '
  '0.020*"gmat" + 0.016*"hours" + 0.014*"roles" + 0.012*"people" + '
  '0.012*"prep" + 0.010*"advice"'),
 (3,
  '0.018*"time" + 0.018*"consulting" + 0.017*"post" + 0.015*"experience" + '
  '0.012*"business" + 0.011*"people" + 0.011*"work" + 0.010*"programs" + '
  '0.010*"industry" + 0.010*"online"'),
 (4,
  '0.035*"work" + 0.019*"school" + 0.016*"people" + 0.013*"class" + '
  '0.013*"career" + 0.011*"students" + 0.011*"pretty" + 0.010*"time" + '
  '0.009*"experience" + 0.009*"summer"'),
 (5,
  '0.022*"people" + 0.020*"consulting" + 0.014*"pro

In [6]:
from gensim.models import CoherenceModel
print(lda_model.log_perplexity(bow_corpus))

-6.736242316900213


In [7]:
doc_list=[]
topic_list=[]
weight_list=[]

i=0
for doc in processed_docs:
    doc_list.append(doc)
    doc=dictionary.doc2bow(doc)
    evaluated_doc=lda_model[doc][0]
    highest_score=sorted(evaluated_doc,key=lambda x: x[1], reverse=True)[0]
    topic_list.append(highest_score[0])
    weight_list.append(highest_score[1])
    i=i+1
topic1={'topic':topic_list}
topic = pd.DataFrame(topic1)  
weight1={'weight': weight_list}
weight = pd.DataFrame(weight1)  

In [8]:
result = result.to_frame()

In [9]:
result = result.assign(topic=topic.values)
result = result.assign(weight=weight.values)

In [10]:
print(result.head())

                                                0  topic    weight
0                  How to submit a profile review      1  0.435085
1                                R2 Status Thread      0  0.136094
2         Scholarship negotiation success stories      2  0.746260
3  Columbia ($$$) vs Wharton for entrepreneurship      1  0.741920
4            Laptop recommendations for b-school?      2  0.607141


In [11]:
import pyLDAvis.gensim

In [ ]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,bow_corpus, dictionary)
pyLDAvis.show(vis)

C:\Users\Upendra\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [14/Jun/2019 18:21:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2019 18:21:35] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2019 18:21:35] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2019 18:21:35] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2019 18:21:35] code 404, message Not Found
127.0.0.1 - - [14/Jun/2019 18:21:35] "GET /favicon.ico HTTP/1.1" 404 -
